## Create Pre-Import XML file

The OpenStreetMapping data file for San Francisco is 846MB which is really big. There are 11 million records. Most of these records are empty nodes which represent points on the earth.


There is no descriptive information for these nodes therefore they are not needed for our data consistency and uniformity checks. To make the data easier to process, we will filter out these empty nodes which do not have descriptive information.

First we store the source and target OSM XML filenames:


In [2]:
# Tiny File
#ORIGINAL_FILE = '/Users/markmavromatis/Downloads/san-francisco_california_tiny.osm'

# Medium File
#ORIGINAL_FILE = '/Users/markmavromatis/Downloads/san-francisco_california_medium.osm'

# Large File
#ORIGINAL_FILE = '/Users/markmavromatis/Downloads/san-francisco_california_large.osm'


# Original File
ORIGINAL_FILE = '/Users/markmavromatis/Downloads/san-francisco_california.osm'

# Output file
FINAL_OXM_XML_FILE = "/Users/markmavromatis/Downloads/san-francisco_california_filtered.osm"

# Parsing errors file
FINAL_OXM_ERRORS_FILE = "/Users/markmavromatis/Downloads/san-francisco_california_errors.txt"


## Phone Number Standardization

Phone numbers formatting is all over the place. We should standardize phone numbers where possible to preserve a format. Our preferred format:

(###) ###-####

Since all phone numbers are in the United States, we can remove the US country code (+1) if present.

In [3]:
import re

# Use regex to extract phone number from a field and standardize it.
# Regex found here: http://www.diveintopython.net/regular_expressions/phone_numbers.html

PHONE_PATTERN_REGEX = re.compile(r'(\d{3})\D*(\d{3})\D*(\d{4})\D*(\d*)$')


# Returns two values:
# 1) True/False Whether the standardization was successful
# 2) Standardized value (if successful)
# 3) Error
def standardize_phone_number(original_phone):

    was_successful = False
    error_message = ""
    new_phone = original_phone
    try:
        
        phone_parsing_result = PHONE_PATTERN_REGEX.search(original_phone)
        if phone_parsing_result == None:
            raise ValueError('Unable to parse phone number: ' + original_phone)
        new_phone_numbersonly = "".join(phone_parsing_result.groups())

        # Remove country code (if included)            
        if new_phone_numbersonly[0] == '1':
            new_phone_numbersonly = new_phone_numbersonly[1:]

        
        if len(new_phone_numbersonly) != 10:
            
            raise ValueError('Invalid # of phone # digits for number: {}'.format(original_phone))

        #Format the number as (<area code>) <prefix>-<suffix>
        new_phone = "(" + new_phone_numbersonly[0:3] + ") " + new_phone_numbersonly[3:6] + "-" + new_phone_numbersonly[6:10]
        was_successful = True
        
    except ValueError as ve:
        error_message = ve.args[0]
        print("Unable to standardize phone number: ({})".format(original_phone))
    return was_successful, new_phone, error_message

In [4]:
# Test phone numbers
print(standardize_phone_number("211 111 1111"))
print(standardize_phone_number("223 456 7890"))
print(standardize_phone_number("(223) 456 7890"))
print(standardize_phone_number("(223) 456-7890"))
print(standardize_phone_number("223-456-7890"))
print(standardize_phone_number("1 223-456-7890"))
print(standardize_phone_number("+1 223-456-7890"))
#Error
print(standardize_phone_number("+1 22-456-7890"))








(True, '(211) 111-1111', '')
(True, '(223) 456-7890', '')
(True, '(223) 456-7890', '')
(True, '(223) 456-7890', '')
(True, '(223) 456-7890', '')
(True, '(223) 456-7890', '')
(True, '(223) 456-7890', '')
Unable to standardize phone number: (+1 22-456-7890)
(False, '+1 22-456-7890', 'Unable to parse phone number: +1 22-456-7890')


## Address Standardization

1) Street Names 

Addresses include different abbreviations for street types. We will standardize them as much as possible to be consistent.


In [5]:


street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# Standardize street types to make street names consistent where possible
# Flag non-standard street names as these may be due to data entry errors.
def standardize_street_name(street_name):

    
    was_successful = False
    error_message = ""
    
    expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Circle", 
            # Added by Mark
            "Highway", "Path", "Terrace", "Way"]

    exception_streets = ["El Camino Real", "Broadway"]

    # Map incomplete street types to their ideal value
    # Ave/Ave. -> Avenue, St -> Street, etc.
    mapping = { 
            "st": "Street",
            "rd": "Road",
            "ave" : "Avenue",
            # Added by Mark
            # A few entries have 'avenue' instead of 'Avenue'
            # This is also true of 'street' 
            # To 'fix' the capitalization, perhaps better to do it outside but this is an easy way.
            "avenue": "Avenue",
            "blvd" : "Boulevard",
            "dr" : "Drive",
            "rd" : "Road",
            "st" : "Street",
            "street" : "Street",
            "hwy" : "Highway"
    }

    # If street name is one of a handful of exceptions, do not attempt to identify street type.
    # E.g. El Camino Real
    if street_name in exception_streets:
        was_successful = True
    else:
        m = street_type_re.search(street_name)
        if m:
            street_type = m.group()
            final_street_type = street_type

            # Convert to lower case and strip trailing period (if exists)
            lookup_street_type = street_type.lower()
            if lookup_street_type[len(lookup_street_type) - 1] == ".":
                lookup_street_type = lookup_street_type[0:len(lookup_street_type) - 1]

            # Check if "street type" needs to be mapped to a different value.
            if lookup_street_type in mapping:
                # Replace original street type with new 'standard' value. 
                street_name = street_name[0:len(street_name) - len(street_type)] + mapping[lookup_street_type]
                # Update street type to new value, so we can check it against expected street types.
                street_type = mapping[lookup_street_type]

            # If the street type is not "expected", log an error and continue.
            if street_type not in expected:
                was_successful = False
                error_message = "Unable to standardize street name: {}".format(street_name)
            else:
                was_successful = True

    return was_successful, street_name, error_message

           

In [6]:
print(standardize_street_name("A Ave."))
print(standardize_street_name("A Ave"))
print(standardize_street_name("A Avenue"))



(True, 'A Avenue', '')
(True, 'A Avenue', '')
(True, 'A Avenue', '')


2) Standardize Postal Codes

Postal codes exist in multiple formats in the OpenStreetMap data:
    e.g. #####, #####-####, <2-letter state abbreviation> 94404

Since the last 4-digits are not very useful for this exercise, we will standardize the postal code as a 5-digit base zip code.


In [7]:
# Standardize postal code in 5-digit format
postal_code_re = re.compile('\d{5}([ \-]\d{4})?')

# Standardize street types to make street names consistent where possible
# Flag non-standard street names as these may be due to data entry errors.
def standardize_postal_code(input_postal_code):
    
    was_successful = False
    error_message = ""
    output_postal_code = ""
    
    try:
        # Extract postal code
        output_postal_code = postal_code_re.search(input_postal_code).group()
        # Only keep first 5 digits
        output_postal_code = output_postal_code[0:5]
        was_successful = True
    except Exception as e:
        was_successful = False
        error_message = "Unsupported postal code format: {}".format(input_postal_code)
        output_postal_code = input_postal_code

    return was_successful, output_postal_code, error_message

## Insert Contact Prefix for Email/Fax/Phone

There is an official 'contact:' prefix tax but this is seldom used. More information can be found here.

http://wiki.openstreetmap.org/wiki/Key:contact

It is useful for grouping together contact details. Let's insert it if the tags are missing.



In [8]:
def is_contact_tag(tag_key):
    return tag_key == 'phone' or tag_key == 'email' or tag_key == 'fax'

In [9]:
DATA_SUBSTITUTIONS = {"3739601815" : [
        # This street tag contained unicode values that were causing problems in my script
        # It looks like corrupted data (with longitude / latitude coordinates mixed in)
        # Let's replace this tag to prevent import errors.
        {
        "tag_name" : "addr:street",
        "tag_value" : "* Removed by pre-import script *"
        }]}

In [10]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Create new XML file with data cleansing changes
"""
import xml.etree.cElementTree as ET
import pprint

PHONE_FIELDS = ['phone','contact:phone','fax','contact:fax']

def cleanse_xml_file(infile, outfile):
    tags = {}
    original_node_way_count = 0
    filtered_node_way_count = 0

    errorsfile = open(FINAL_OXM_ERRORS_FILE, 'w')
    errorsfile.write("ID,ErrorType,SourceValue,ErrorMessage\n")
    with open(outfile, 'w') as f:
        # Create opening XML tag in our new file
        f.write("<osm>")
        for event, elem in ET.iterparse(infile, events = ('start',)):
            tag = elem.tag

            # Check for nodes with child tags
            if elem.tag == "node" or elem.tag == "way":
                original_node_way_count += 1
                for tag in elem.iter("tag"):
                    key_name = tag.attrib['k']
                    key_value = tag.attrib['v']
                    
                    # Replace value if necessary
                    record_id = elem.attrib['id']
                    if record_id in DATA_SUBSTITUTIONS:
                        for row in DATA_SUBSTITUTIONS[record_id]:
                            if key_name == key_name:
                                key_value = row['tag_value']                        
                        

                    final_key_name = ""
                    final_key_value = ""

                    # Check for contact: tag
                    if is_contact_tag(key_name):
                        final_key_name = "contact:" + key_name
                    

                    # Rename k='address' tag
                    # In our final schema, we will rename the k='addr:___' tags to an address dictionary.
                    # To prevent collisions with this tag, we will rename it to 'addr:address'
                    if key_name == "address":
                        final_key_name = "addr:address"

                    # Cleanse phone number field
                    if key_name in PHONE_FIELDS:
                        was_successful, final_key_value, error_message = standardize_phone_number(key_value)
                        if not was_successful:
                            errorsfile.write("{}|Phone|{}\n".format(elem.attrib['id'], key_value))

                    # Cleanse street name field
                    if key_name == "addr:street":
                        was_successful, final_key_value, error_message = standardize_street_name(key_value)
                        if not was_successful:
                            errorsfile.write("{}|Address|{}\n".format(elem.attrib['id'], key_value))

                    # Standardize postal code field
                    if key_name == "addr:postcode":
                        was_successful, final_key_value, error_message = standardize_postal_code(key_value)
                        if not was_successful:
                            errorsfile.write("{}|PostalCode|{}\n".format(elem.attrib['id'], key_value))

                    
                    # Overwrite key value with new value
                    if final_key_value != "":
                        tag.attrib['v'] = final_key_value
                    
                    # Overwrite key name with new value
                    if final_key_name != "":
                        tag.attrib['k'] = final_key_name

                        
                filtered_node_way_count += 1
                f.write(ET.tostring(elem))
        # Create closing XML tag in our new file
        f.write("</osm>")
        errorsfile.close()

        print("Total # of Nodes and Ways: {:,}".format(original_node_way_count))
        print("Filtered # of Nodes and Ways: {:,}".format(filtered_node_way_count))
        print("Errors File: {}".format(FINAL_OXM_ERRORS_FILE))


    return tags

cleanse_xml_file(ORIGINAL_FILE, FINAL_OXM_XML_FILE)
print("Created new filtered nodes file: " + FINAL_OXM_XML_FILE)

Unable to standardize phone number: (415-397-BROS)
Unable to standardize phone number: (+49)
Unable to standardize phone number: (885-2222)
Unable to standardize phone number: (415 242 960)
Unable to standardize phone number: (415 409 FARM)
Unable to standardize phone number: (http://www.pastapastaco.com/)
Unable to standardize phone number: (B Street & Vine)
Unable to standardize phone number: (650368384)
Unable to standardize phone number: (415-252-855)
Unable to standardize phone number: (fire)
Unable to standardize phone number: (+1-510-336-BAGS)
Unable to standardize phone number: (+1-510-859-PIES)
Unable to standardize phone number: (667-7005)
Unable to standardize phone number: (15-358-1220)
Unable to standardize phone number: (yes)
Unable to standardize phone number: (+1 415 759 TOYS)
Total # of Nodes and Ways: 4,412,192
Filtered # of Nodes and Ways: 4,412,192
Errors File: /Users/markmavromatis/Downloads/san-francisco_california_errors.txt
Created new filtered nodes file: /User